In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GraphSAGE

import importlib

import modules.training as training
importlib.reload(training)

<module 'training' from 'c:\\Users\\csaba\\Documents\\Coding\\git_own\\thesis_coding\\training.py'>

In [2]:
device = utils.set_seeds_and_device() 
dataset,data = training.load_dataset('Cora', "../training_data/datasets")

NameError: name 'utils' is not defined

In [ ]:
# Define model
model = GraphSAGE(
    in_channels=dataset.num_features,
    hidden_channels=64,
    num_layers=2,
    out_channels=dataset.num_classes,
    dropout=0.5
).to('cpu')

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Supervised training loop using NLL loss
for epoch in range(1, 201):
    model.train()
    out = model(data.x, data.edge_index)
    log_probs = F.log_softmax(out, dim=1)
    loss = F.nll_loss(log_probs[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# Evaluation
model.eval()
pred = out.argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).sum() / data.test_mask.sum()
print(f'Test Accuracy: {acc:.4f}')


Epoch 20, Loss: 0.0003
Epoch 40, Loss: 0.0000
Epoch 60, Loss: 0.0001
Epoch 80, Loss: 0.0000
Epoch 100, Loss: 0.0000
Epoch 120, Loss: 0.0000
Epoch 140, Loss: 0.0000
Epoch 160, Loss: 0.0000
Epoch 180, Loss: 0.0000
Epoch 200, Loss: 0.0000
Test Accuracy: 0.7360


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Prepare embeddings and labels
X = embeddings.cpu().numpy() if hasattr(embeddings, 'cpu') else embeddings
y = data.y.cpu().numpy() if hasattr(data.y, 'cpu') else data.y

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(),
    "MLP": MLPClassifier(max_iter=500)
}

results = {}

for name, model in models.items():
    try:
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"{name} Test accuracy: {acc:.4f}")
        print(classification_report(y_test, y_pred))
        results[name] = acc
    except Exception as e:
        print(f"{name} failed: {e}")

# Summary table
results_df = pd.DataFrame(list(results.items()), columns=["Model", "Test Accuracy"])
print("\nModel Comparison Summary:")
print(results_df.sort_values(by="Test Accuracy", ascending=False).to_string(index=False))